# Pràctica II - Perceptron amb Scikit-learn

En la sessió anterior hem implementat i entrenat un perceptró des de zero, analitzant pas a pas el seu funcionament intern. Ara, donarem un pas més i treballarem amb la implementació ja disponible a la llibreria scikit-learn, que ens permetrà concentrar-nos en el procés d’aplicació del model a un conjunt de dades real.

Per tal d’il·lustrar-ho, farem servir el problema del Titanic, que ja coneixeu del curs anterior. L’objectiu és predir si un passatger va sobreviure o no al naufragi a partir d’algunes de les seves característiques (edat, sexe, classe del bitllet, etc.).

Tasques a realitzar

1. Carregar i preparar les dades
   - Importar el conjunt de dades del Titanic.
   - Seleccionar les variables més rellevants i realitzar els preprocessaments necessaris (tractament de valors NaNs, codificació de variables categòriques i normalització).
   - Dividir les dades en conjunts d’entrenament i validació.

2. Definir i entrenar el model
   - Utilitzar la classe Perceptron de ``sklearn.linear_model``.
   - Ajustar el model amb les dades d’entrenament.
   - Utilitzar el perceptró implementat la sessió anterior per comparar resultats.

3. Avaluar el rendiment
   - Calcular la *accuracy* sobre les dades de validació.
   - Analitzar la matriu de confusió per entendre millor els encerts i errors del model.
   - Comparar els resultats de ``scikit-learn`` amb la vostra implementació prèvia.

In [32]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from PerceptronP import PerceptronP

Podeu trobar les dades del Titanic a [Kaggle](https://www.kaggle.com/competitions/titanic/overview).

In [28]:
df = pd.read_csv('train.csv')

## Neteja de dades

### Codificació de variables categòriques

Farem servir la codificació *one-hot* per a les variables categòriques, ``pandas`` té una funció que ens ho facilita: ``get_dummies()``.

In [7]:
categoricals = ['Sex', 'Embarked', 'Pclass']

df = pd.get_dummies(df, columns=categoricals)

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,False,True,False,False,True,False,False,True
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,True,False,True,False,False,True,False,False
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,True,False,False,False,True,False,False,True
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,True,False,False,False,True,True,False,False
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,False,True,False,False,True,False,False,True


### Eliminació de les columnes sense informació

Hi ha algunes columnes que no aporten informació rellevant per a la predicció, com ara el nom del passatger o el número del bitllet. Les hem d'eliminar.

In [8]:
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

,Survived,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,0,22.0,1,0,7.2500,False,True,False,False,True,False,False,True
1,1,38.0,1,0,71.2833,True,False,True,False,False,True,False,False
2,1,26.0,0,0,7.9250,True,False,False,False,True,False,False,True
3,1,35.0,1,0,53.1000,True,False,False,False,True,True,False,False
4,0,35.0,0,0,8.0500,False,True,False,False,True,False,False,True


### Normalitzam les dades

Hem de normalitzar les dades per tal que totes les característiques tinguin la mateixa escala. Això és especialment important per a algorismes com el perceptró, que són sensibles a la magnitud de les característiques.

In [9]:
scaler = StandardScaler()

X = df.drop(columns=['Survived'])
y = df['Survived']

# Ajustar i transformar X
X_scaled = scaler.fit_transform(X)

### Tractament de valors NaN

Els valors NaN els hem de tractar abans d'entrenar el model. Podem optar per eliminar les files amb valors NaN o bé imputar-los amb la mitjana, mediana o moda de la columna. Pandas ens ofereix diverses funcions per fer-ho. Per exemple, podem utilitzar ``fillna()`` per imputar valors, o ``dropna()`` per eliminar files amb valors NaN.

In [11]:
# print(df.isna().sum())

df['Age'] = df['Age'].fillna(df['Age'].median())

In [33]:
# Tractament de dades complet

df = pd.read_csv('train.csv')

# 1. Codificació de variables categòriques
categoricals = ['Sex', 'Embarked', 'Pclass']
df = pd.get_dummies(df, columns=categoricals, drop_first=True)

# 2. Eliminació de columnes sense informació
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

# 3. Tractament de Nans
# print(df.isna().sum())

df['Age'] = df['Age'].fillna(df['Age'].median())

# 4. Normalització
X = df.drop(columns=['Survived'])
y = df['Survived']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Divisió del conjunt de dades

Separarem les dades en un conjunt d'entrenament i un de validació. Això ens permetrà avaluar el rendiment del model en dades que no ha vist durant l'entrenament. S'utilitza la funció ``train_test_split()`` de ``sklearn.model_selection``.

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

## Entrenament i avaluació del model

Hem d'entrenar el perceptró amb les dades d'entrenament i després avaluar-lo amb les dades de validació. Hem de fer dos entrenaments, un amb la nostra implementació i un altre amb la de ``scikit-learn``. Per emprar la de ``scikit-learn``, utilitzarem la classe ``Perceptron`` de ``sklearn.linear_model``.

In [35]:
model = Perceptron(max_iter=1000, tol=1e-3, random_state=42)
meu_model = PerceptronP(eta=0.01, n_iter=10)

y_train_perceptron = y_train.replace(0, -1)
y_test_perceptron = y_test.replace(0, -1)

model.fit(X_train, y_train)
meu_model.fit(X_train, y_train_perceptron)

y_pred = model.predict(X_test)
meu_y_pred = meu_model.predict(X_test)

meu_y_pred_corrected = np.where(meu_y_pred == -1, 0, 1)

accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy: .4f}")

accuracy_my = (meu_y_pred_corrected == y_test).mean()
print(f"Accuracy meu perceptró: {accuracy_my:.4f}")

Accuracy:  0.7430
Accuracy meu perceptró: 0.7263
